In [1]:
import re
import os
import requests
import subprocess as sp
import pandas as pd
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [17]:
#now final list of files downloaded
data = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/u_triggers_with_probs2.csv"))
with open("/home/ravioli/astro/git/fermi_grbs/u_triggers_with_probs.csv", "w") as f:
    for i in range(len(data)):
        if (data['healpix'].values[i] != 2048 and data['healpix'].values[i] != 'None') or (data['locprob'].values[i] != 2048 and data['healpix'].values[i] != 'None'):
            f.write(f"{data['url'].values[i]},{data['GRBName'].values[i]},{data['Trig_no'].values[i]},{data['healpix'].values[i]},{data['locprob'].values[i]}\n")
    f.close()

In [12]:
#redownload some files with code 31744
downloaded = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/u_triggers_with_probs.csv"))
os.chdir("/home/ravioli/astro/git/fermi_grbs/data/")
for i, url in enumerate(downloaded['url']):
    if downloaded['healpix'][i] != 2048 and downloaded['healpix'][i] != 0:
        healpix = f"{url}glg_healpix_all_{downloaded['GRBName'][i]}.fit"
        r_healpix = int(os.system(f"timeout 20s wget -nc {healpix}"))
        print(f"healpix {downloaded['Trig_no'][i]} {downloaded['healpix'][i]} {r_healpix}")
        downloaded.loc[i, 'healpix'] = r_healpix
    elif downloaded['locprob'][i] != 2048 and downloaded['locprob'][i] != 0:
        locprob = f"{url}glg_locprob_all_{downloaded['GRBName'][i]}.fit"
        r_locprob = int(os.system(f"timeout 20s wget -nc {locprob}"))
        print(f"locprob {downloaded['Trig_no'][i]} {downloaded['locprob'][i]} {r_locprob}")
        downloaded.loc[i, 'locprob'] = r_locprob
pd.DataFrame.to_csv(downloaded, "/home/ravioli/astro/git/fermi_grbs/u_triggers_with_probs2.csv", index=False)

locprob 513398525 31744 0
locprob 516194820 31744 0
locprob 525693395 31744 0
healpix 531353355 1024 2048
locprob 531640821 31744 0
healpix 531720094 31744 2048
healpix 531786044 31744 2048
healpix 533223252 31744 2048
healpix 534599360 31744 2048
locprob 538812207 31744 0
locprob 539806855 31744 0
locprob 540659650 31744 0
healpix 540732857 31744 0
healpix 540810844 31744 2048
locprob 542274195 31744 0
healpix 542425558 31744 0
healpix 544531428 31744 0
healpix 544685682 31744 2048
healpix 544799945 31744 2048
healpix 544948480 31744 0
healpix 545291239 31744 2048
healpix 545553990 31744 2048
healpix 545609416 31744 0
locprob 545726651 1024 0
locprob 546486885 1024 2048
locprob 547264230 31744 0
healpix 548234666 31744 0
locprob 549533811 31744 2048
locprob 550330712 31744 0
locprob 550655469 31744 0
healpix 552038584 31744 0
locprob 552049919 31744 0
locprob 552878923 1024 0
healpix 553879512 31744 0
healpix 553996158 31744 0
healpix 554603145 31744 0
locprob 554998194 31744 2048
loc

In [5]:
#find entries which have healpix or locmap
data = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/data_status.csv"))
triggers = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/u_triggers.csv"))
with open("/home/ravioli/astro/git/fermi_grbs/u_triggers_with_probs.csv", "w") as f:
    for i in range(len(data)):
        if (data['healpix'].values[i] != '2048' and data['healpix'].values[i] != 'None') or (data['locmap'].values[i] != '2048' and data['healpix'].values[i] != 'None'):
            url = triggers['url'].values[np.where(triggers['trigger_no'].values == data['Trig_no'].values[i])[0][0]]
            f.write(f"{url},{data['GRBName'].values[i]},{data['Trig_no'].values[i]},{data['healpix'].values[i]},{data['locmap'].values[i]}\n")
    f.close()

In [8]:
# #downloading all healpix and locprob files
# df = pd.DataFrame(pd.read_csv("/home/ravioli/astro/git/fermi_grbs/u_triggers.csv"))

# with open("data_status.csv", "a") as f:
#     os.chdir("/home/ravioli/astro/git/fermi_grbs/data/")
#     for url, trigger in zip(df['url'].values, df['trigger_no'].values):
#         if url != "None":
#             print(url)
#             grb = url.split("/")[-3]
#             healpix = f"{url}glg_healpix_all_{grb}.fit"
#             locprob = f"{url}glg_locprob_all_{grb}.fit"
#             r_healpix = int(os.system(f"timeout 20s wget -nc {healpix}"))
#             r_locprob = int(os.system(f"timeout 20s wget -nc {locprob}"))
#             if not (r_healpix in [0,2048,256] or r_locprob in [0,2048,256]):
#                 print("Downloading again")
#                 print(url)
#                 r_healpix = os.system(f"timeout 20 wget -nc {healpix}")
#                 r_locprob = os.system(f"timeout 20 wget -nc {locprob}")
#             f.write(f"{grb},{trigger},{r_healpix},{r_locprob}\n")
#             # print(f"{grb},{trigger},{r_healpix},{r_locprob}")
#         else:
#             print(url)
#             f.write(f"{None},{trigger},{None},{None}\n")

# f.close()

In [8]:
# #getting data products link from GCN
# basepath = "https://gcn.gsfc.nasa.gov/other/"
# with open("u_triggers.txt", "a") as f:
#     for trigger in u_triggers:
#         try:
#             r = requests.get(f"{basepath}/{trigger}.fermi")
#             url = os.path.dirname(re.search("(?P<url>http?://[^\s]+)", r.text).group("url"))+"/"
#             f.write(f"{url}\t{trigger}\n")
#         except:
#             print(f"Trigger {trigger} not found")
#             f.write(f"{None}\t{trigger}\n")
# f.close()

Trigger 508930304 not found
Trigger 553789309 not found
Trigger 558642187 not found
Trigger 567431057 not found
Trigger 647120007 not found
Trigger 687017830 not found
